In [1]:
%load_ext nb_mypy

Version 1.0.5


In [2]:
from loguru import logger
from s3path import S3Path
from datetime import datetime

from sm_pipelines_oo.config_loader.implementations.mock_config_loader import MockConfigLoader
from sm_pipelines_oo.pipeline import PipelineFacade, DevPipelineFacade
from sm_pipelines_oo.shared_config_schema import Environment

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thomas-22/.config/sagemaker/config.yaml


In [3]:
_PROJECT_BUCKET = S3Path('/smp-oo-example')
_DATA_PREFIX = 'examples/data/'
output_filename = f'output-{datetime.now()}'

env: Environment = 'dev'
_shared_config_dict={
    'project_name': 'test',
    'project_version': '0',
    'region': 'us-east-1',
    'project_bucket_name': _PROJECT_BUCKET.bucket,
    'role_name': 'sagemaker_pipelines_role'
}
_fw_processor_config_dict = {
    'step_name': 'preprocessing',
    'step_factory_class': 'FrameworkProcessor',
    'processor_init_config': {
        'framework_version': '0.23-1',
        'estimator_cls_name': 'SKLearn',
        'instance_count': 1,
        'instance_type': 'ml.m5.xlarge',
        'env': {'OUTPUT_FILENAME': output_filename},
    },
    'processor_run_config': {
        'code': 'preprocess.py',
        'source_dir': 'worker_code/preprocess',
        'inputs': {
            'input_1': (_PROJECT_BUCKET / _DATA_PREFIX / "input_1" ).as_uri(),
        },
        'outputs': {
            'output_1': (_PROJECT_BUCKET / _DATA_PREFIX / "output_1").as_uri(),
        },
    },
    'shared_config': _shared_config_dict
}
_mock_config_loader = MockConfigLoader(
    shared_config_dict=_shared_config_dict,
    step_configs_dicts=[_fw_processor_config_dict,]
)

In [4]:
pipeline = DevPipelineFacade(
    env=env,
    # Use different configs for testing
    custom_config_loader=_mock_config_loader,
)


INFO:botocore.tokens:Loading cached SSO token for ml
/home/thomas-22/.cache/pypoetry/virtualenvs/sm-pipelines-oo-examples-yainlQgE-py3.10/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [5]:
pipeline.create_and_start_pipeline_from_definition()

INFO:botocore.tokens:Loading cached SSO token for ml
INFO:sagemaker.processing:Uploaded worker_code/preprocess to s3://sagemaker-us-east-1-338755209567/test-v0/code/592ac4c2e2e67be053a8e95236cafc40/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-338755209567/test-v0/code/bc2536a25d34e1ecae5238f42f4207c2/runproc.sh
/home/thomas-22/.cache/pypoetry/virtualenvs/sm-pipelines-oo-examples-yainlQgE-py3.10/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
2024-03-15 10:08:44.321 | INFO     | sm_pipelines_oo.pipeline:export_pipeline_definition_to_s3:82 - Uploaded pipeline definition to s3://smp-oo-example/pipeline_definitions/test-v0.json

An error occurred (ValidationException) when calling the CreatePipeline operation: Pipeline names must be unique within an AWS account and region. Pipeline with name (test-v0

In [7]:
from smp_oo_examples.validation import check_output

check_output(
    output_path_s3=_PROJECT_BUCKET / _DATA_PREFIX / "output_1" / output_filename,
    wait_time_in_minutes=20,
)

   a   b   c
0  2   8  14
1  4  10  16
2  6  12  18
Success


In [8]:
print(datetime.now())

2024-03-15 10:29:15.504114
